<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=343891ed016bb7464a5b2db99d881e19eefc1234abda566c7ea18d75cfa86ea7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 12:25:09
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -88.89 K (-0.63%)
Current PnL: -19.01 L (-12.79%)
CY Booked + Current PnL: -7.57 L (-5.09%)
-------------------
Total profit:  1.48 L
Total loss:  -20.49 L
-------------------
Total Booked + Current PnL: 19.41 L (15.95%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.88 L (60.38%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 8.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.61,61.0,H-MC,3.63,196130.0,31460.0,11866.0,-1.78,19.10,6.05,26.31,89.0,2.65,1.40,28.61,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.01,41.0,M-SC,3.10,84895.0,-15882.0,15977.0,-1.33,-15.76,18.82,0.09,245.0,-0.99,0.60,11.53,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.22,110537.0,10033.0,19079.0,-1.13,9.98,17.26,28.96,79.0,0.53,0.79,47.34,MH,ATH,METALS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.66,215436.0,5070.0,23720.0,-0.69,2.41,11.01,13.68,37.0,0.21,1.53,19.33,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.49,85248.0,-28301.0,68352.0,3.35,-24.92,80.18,35.27,268.0,-0.41,0.61,108.65,XR,NTT,HOTELS
7,ATULAUTO,844.0,3824.32,59.0,M-SC,5.33,122342.0,-25026.0,76843.0,3.28,-16.98,62.81,35.16,236.0,-0.33,0.87,24.96,XY24,NTT,AUTO
39,INDUSINDBK,1800.0,245.94,44.0,L-MC,12.30,42442.0,-34764.0,60157.0,1.40,-45.03,141.74,32.89,258.0,-0.58,0.30,16.88,XR,NTT,BANKS
66,SIS,528.0,2074.61,36.0,H-SC,4.84,87758.0,-23274.0,46880.0,0.98,-20.96,53.42,21.26,156.0,-0.50,0.62,18.57,OX40N,NTT,MISC
67,SONACOMS,1006.0,-40.39,40.0,M-SC,11.70,74226.0,-26933.0,101823.0,0.84,-26.62,137.18,74.03,202.0,-0.26,0.53,4.67,AR,BTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-64.90,52.0,H-MC,3.77,249685.0,-52066.0,213481.0,-3.56,-17.25,85.50,53.50,116.0,-0.24,1.78,10.90,XY24,NTT,FMCG
6,ASIANTILES,137.0,6900.00,57.0,L-SC,5.21,76792.0,-17018.0,93632.0,-2.34,-18.14,121.93,81.67,269.0,-0.18,0.55,48.71,XR,NTT,CERAMICS
61,SAMMAANCAP,326.0,-171.59,58.0,M-SC,4.45,82740.0,-19470.0,112857.0,-2.31,-19.05,136.40,91.37,208.0,-0.17,0.59,34.31,XY25,NTT,FINANCE
10,BANDHANBNK,400.0,-14.68,39.0,H-SC,4.69,214092.0,-64467.0,321909.0,-2.10,-23.14,150.36,92.42,151.0,-0.20,1.52,23.73,XY24,NTT,BANKS
64,SHALBY,327.0,1225.36,75.0,M-SC,8.06,180215.0,-928.0,46405.0,-2.10,-0.51,25.75,25.10,235.0,-0.02,1.28,44.42,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1225.36,75.0,M-SC,8.06,180215.0,-928.0,46405.0,-2.10,-0.51,25.75,25.10,235.0,-0.02,1.28,44.42,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-20.77,46.0,H-MC,2.07,136323.0,535.0,42219.0,-1.03,0.39,30.97,31.48,107.0,0.01,0.97,12.56,X40,ATH,INSURANCE
23,DABUR,735.00,-7.62,39.0,H-MC,6.90,250260.0,2192.0,106210.0,-1.16,0.88,42.44,43.70,102.0,0.02,1.78,15.46,XY24,BTT,FMCG
29,HAVELLS,2069.16,-0.94,52.0,H-MC,2.02,245419.0,-2312.0,77381.0,-0.51,-0.93,31.53,30.30,92.0,-0.03,1.75,12.23,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.02,54.0,H-MC,3.78,202425.0,1605.0,67165.0,-0.29,0.80,33.18,34.25,80.0,0.02,1.44,6.04,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,9.25,120069.0,-22410.0,41928.0,-0.67,-15.73,34.92,13.70,163.0,-0.53,0.85,41.71,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.01,41.0,M-SC,3.10,84895.0,-15882.0,15977.0,-1.33,-15.76,18.82,0.09,245.0,-0.99,0.60,11.53,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,54.0,H-SC,0.87,223209.0,-46458.0,82788.0,-1.97,-17.23,37.09,13.47,138.0,-0.56,1.59,12.90,XY24,NTT,PAINTS
18,CERA,9475.0,-21.72,38.0,H-SC,2.10,142876.0,-33027.0,75053.0,-1.43,-18.78,52.53,23.89,149.0,-0.44,1.02,23.39,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.53,42.0,H-MC,7.11,104870.0,-26965.0,69193.0,-1.56,-20.45,65.98,32.03,98.0,-0.39,0.75,17.46,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,48.36,69.0,M-MC,8.92,234364.0,9402.0,158243.0,0.45,4.18,67.52,74.53,192.0,0.06,1.67,37.78,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.61,61.0,H-MC,3.63,196130.0,31460.0,11866.0,-1.78,19.10,6.05,26.31,89.0,2.65,1.40,28.61,X40N,NTT,AC
84,VOLTAS,1530.00,-2.06,45.0,H-MC,2.84,205500.0,13758.0,24002.0,-0.45,7.18,11.68,19.69,99.0,0.57,1.46,14.46,XY25,NTT,AC
17,CAMS,5211.76,-2.06,56.0,H-SC,2.64,111398.0,9394.0,34533.0,-0.88,9.21,31.00,43.06,122.0,0.27,0.79,27.90,X40N,ATH,MISC
23,DABUR,735.00,-7.62,39.0,H-MC,6.90,250260.0,2192.0,106210.0,-1.16,0.88,42.44,43.70,102.0,0.02,1.78,15.46,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,60.0,H-MC,9.22,110537.0,10033.0,19079.0,-1.13,9.98,17.26,28.96,79.0,0.53,0.79,47.34,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.17,48.0,M-SC,1.13,99368.0,7870.0,70879.0,-0.32,8.60,71.33,86.07,223.0,0.11,0.71,44.30,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.97,185206.0,5398.0,107012.0,-0.09,3.00,57.78,62.52,88.0,0.05,1.32,38.11,XR,NTT,BANKS
37,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129813.0,6477.0,127282.0,-0.82,5.25,98.05,108.45,119.0,0.05,0.92,28.78,AR,ATH,MISC
86,WIPRO,420.00,-11.52,49.0,M-LC,5.67,155087.0,4142.0,105320.0,-0.08,2.74,67.91,72.51,53.0,0.04,1.10,8.93,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.39,36.0,H-LC,7.16,293799.0,-22043.0,134707.0,-0.74,-6.98,45.85,35.67,5.0,-0.16,2.09,5.91,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.93,223882.0,-18042.0,81314.0,-0.97,-7.46,36.32,26.15,8.0,-0.22,1.59,5.80,X40,ATH,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-9.52,40.0,H-LC,10.31,253890.0,-3173.0,38312.0,-1.29,-1.23,15.09,13.67,24.0,-0.08,1.81,18.92,XY25,NTT,FMCG
19,CIPLA,1795.00,-21.84,41.0,H-LC,5.29,208637.0,4137.0,37283.0,-1.25,2.02,17.87,20.25,10.0,0.11,1.48,10.93,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.34,50.0,H-LC,1.48,237398.0,-8753.0,87956.0,0.29,-3.56,37.05,32.18,16.0,-0.10,1.69,31.21,X200,ATH,IT
42,ITC,452.0,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,55.0,H-LC,3.33,162185.0,-23910.0,71394.0,0.01,-12.85,44.02,25.51,15.0,-0.33,1.15,19.52,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-11.28,43.0,H-LC,4.54,273148.0,7722.0,49057.0,-1.77,2.91,17.96,21.40,11.0,0.16,1.94,10.92,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.66,215436.0,5070.0,23720.0,-0.69,2.41,11.01,13.68,37.0,0.21,1.53,19.33,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287837.0,-58119.0,132981.0,-0.38,-16.80,46.20,21.64,1.0,-0.44,2.05,1.97,X40,ATH,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.37,54.0,H-LC,7.51,229758.0,-4386.0,53005.0,-0.31,-1.87,23.07,20.77,86.0,-0.08,1.63,4.92,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.93,223882.0,-18042.0,81314.0,-0.97,-7.46,36.32,26.15,8.0,-0.22,1.59,5.80,X40,ATH,IT
83,VBL,671.64,-18.39,36.0,H-LC,7.16,293799.0,-22043.0,134707.0,-0.74,-6.98,45.85,35.67,5.0,-0.16,2.09,5.91,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,55.0,H-LC,3.33,162185.0,-23910.0,71394.0,0.01,-12.85,44.02,25.51,15.0,-0.33,1.15,19.52,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,41.0,H-LC,5.29,208637.0,4137.0,37283.0,-1.25,2.02,17.87,20.25,10.0,0.11,1.48,10.93,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.66,215436.0,5070.0,23720.0,-0.69,2.41,11.01,13.68,37.0,0.21,1.53,19.33,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-14.37,41.0,H-LC,7.63,216269.0,-35499.0,88714.0,-1.01,-14.10,41.02,21.13,27.0,-0.40,1.54,16.33,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287837.0,-58119.0,132981.0,-0.38,-16.80,46.20,21.64,1.0,-0.44,2.05,1.97,X40,ATH,IT
40,INFY,2275.00,-17.80,50.0,H-LC,8.16,320054.0,7166.0,164508.0,0.21,2.29,51.40,54.87,3.0,0.04,2.28,9.09,X40,BTT,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
83,VBL,671.64,-18.39,36.0,H-LC,7.16,293799.0,-22043.0,134707.0,-0.74,-6.98,45.85,35.67,5.0,-0.16,2.09,5.91,X40N,ATH,FMCG
1,ABB,7934.00,-39.09,53.0,H-LC,9.04,254467.0,-7152.0,126368.0,-1.15,-2.73,49.66,45.57,7.0,-0.06,1.81,8.28,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6900.00,57.0,L-SC,5.21,76792.0,-17018.0,93632.0,-2.34,-18.14,121.93,81.67,269.0,-0.18,0.55,48.71,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,4.45,82740.0,-19470.0,112857.0,-2.31,-19.05,136.40,91.37,208.0,-0.17,0.59,34.31,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.49,85248.0,-28301.0,68352.0,3.35,-24.92,80.18,35.27,268.0,-0.41,0.61,108.65,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.36,42.0,H-SC,12.05,87152.0,-13811.0,108339.0,-0.06,-13.68,124.31,93.63,148.0,-0.13,0.62,29.81,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.80,48.0,H-SC,12.66,93585.0,-4395.0,25998.0,-0.92,-4.49,27.78,22.05,152.0,-0.17,0.67,36.03,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1225.36,75.0,M-SC,8.06,180215.0,-928.0,46405.0,-2.10,-0.51,25.75,25.10,235.0,-0.02,1.28,44.42,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3824.32,59.0,M-SC,5.33,122342.0,-25026.0,76843.0,3.28,-16.98,62.81,35.16,236.0,-0.33,0.87,24.96,XY24,NTT,AUTO
71,TANLA,1963.11,-15.40,71.0,H-SC,14.41,226440.0,-44321.0,374283.0,0.66,-16.37,165.29,121.86,133.0,-0.12,1.61,77.14,AR,ATH,IT
81,VAIBHAVGBL,521.00,60.88,61.0,H-SC,6.03,139477.0,-43298.0,154373.0,-1.36,-23.69,110.68,60.77,125.0,-0.28,0.99,24.92,XR,NTT,JEWELLERY
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,4.45,82740.0,-19470.0,112857.0,-2.31,-19.05,136.40,91.37,208.0,-0.17,0.59,34.31,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.25
1,25,43.67
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.06
LC    32.38
MC    23.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.36
X40      14.73
X40N     11.41
XR       11.28
XY25     10.03
AR        8.42
OX40N     7.80
X200      1.69
MH        1.65
X5K       1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.41
H-LC    25.89
H-MC    20.62
M-SC    13.13
M-LC     5.45
M-MC     2.65
L-SC     1.52
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.90,-5.06,39.74
IT,12.99,-15.22,78.05
FINANCE,10.07,-15.98,66.58
MISC,7.07,-15.56,79.81
BANKS,6.07,-16.99,78.53
PAINTS,5.63,-16.67,34.06
ELECTRICAL,5.50,-6.59,43.99
INSURANCE,3.70,-6.14,45.82
AC,3.55,3.01,13.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3113971.0,22
XR,1327121.0,14
AR,1147678.0,9
X40,778536.0,10
X40N,601914.0,9
OX40N,549674.0,10
XY25,429558.0,7
SR,249053.0,2
X5K,119699.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3662525.0,29
M-SC,1399394.0,17
H-MC,1210397.0,15
H-LC,1178408.0,15
M-LC,384921.0,4
M-MC,305769.0,2
L-SC,248261.0,3
L-MC,60157.0,1
L-LC,38194.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1227573.0      6
M-SC       XY24       798237.0      7
H-SC       AR         784990.0      5
           XR         776862.0      7
H-MC       XY24       557381.0      4
H-LC       X40        491497.0      5
M-MC       XY24       305769.0      2
H-SC       X40N       300921.0      4
           OX40N      259339.0      4
           SR         249053.0      2
H-LC       X40N       224995.0      3
H-MC       X40        214316.0      4
H-LC       AR         197762.0      2
H-MC       XY25       167418.0      2
M-SC       AR         164926.0      2
L-SC       XR         161984.0      2
M-LC       XY24       159902.0      2
M-SC       OX40N      134865.0      4
M-LC       X5K        119699.0      1
M-SC       XR         115786.0      2
           XY25       112857.0      1
H-LC       XY25       111089.0      3
H-MC       XR         107012.0      1
M-LC       XR         105320.0      1
H-LC       X200        87956.0      1
L-SC       OX40N       86277.0      1
H-MC       X40N        75998.0      2
M-SC       X40         72723.0      1
H-MC       OX40N       69193.0      1
H-LC       XY24        65109.0      1
H-SC       MH          63787.0      1
L-MC       XR          60157.0      1
L-LC       XY25        38194.0      1
H-MC       MH          19079.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
